* This notebook will pre-process images for training and test
* This will prepare multi-label one-hot-encoding for labels 
* This will resize images to 64x64 and save a batch of 1000 images in a pickle file
* Assumption - Needs following dir structure
* Directory structure - 
   *  ../data - top layer directory for images
   *  ../data/train_v2.csv  - csv file for training images
   *  ../data/train-jpg - dir for training images
   *  ../data/test-jpg -  dir for testing images
   *  ../data/test-jpg-additional - dir for additional testing images

In [ ]:
import os
import pandas as pd
import cv2
from pprint import pprint
import numpy as np
from itertools import chain
import pickle


In [ ]:
"""
data is kept one level up
"""
data_root_folder = os.path.abspath("../data/")
train_root_dir=os.path.join(data_root_folder, 'train-jpg')
train_csv_file = os.path.join(data_root_folder, 'train_v2.csv')

In [ ]:
"""
This routine makes a dictionary of all unique and sorted labels.
The labels are entered as keys with  corresponding indexs as values. 
"""
def make_label_map():

    train_csv_df = pd.read_csv(train_csv_file)

    #print (train_csv_df.head())

    labels = sorted(set(chain.from_iterable([tags.split(" ") for tags in train_csv_df['tags'].values])))

    #print (labels)

    label_map = dict()

    for index, label in enumerate(labels):
        label_map[label] = index

    
    return label_map

label_map = make_label_map()
pprint(label_map)

In [ ]:
"""
This function will read image into an array from image file 
This will resize each image and normalize by dividing with 255.0
"""

def read_image(image_filename, image_resize):
    """
    :param_args: list of arguments
        image_file_name: string
            The image file name
        image_resize: tuple (int, int)
            The resize dimension
    """

    #print image_name
    
    print image_filename
    image_array = cv2.imread(image_filename)

    # convert to RGB
    image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
    # resize
    image_array = cv2.resize(image_array, image_resize)
    # normalize
    image_array = image_array.astype(float) / 255.0
    return image_array

#unit-test
data_root_folder = os.path.abspath("../data/")
dir_path = os.path.join(data_root_folder, 'train-jpg')
test_image_name = os.path.join(dir_path, 'train_0.jpg')

test_image_resize = (64, 64)
test_image = read_image(test_image_name, test_image_resize)
print test_image.shape


In [ ]:
"""
This function is one hot encoder for multiple labels in tags
""" 
def multi_label_one_hot(tags, label_map):
    ml_one_hot = np.zeros(len(label_map))
    for label in tags.split(' '):
        if label in label_map:
            ml_one_hot[label_map[label]] = 1
    return ml_one_hot

# unit-test 
test_tags = "haze primary"
test_label_map = make_label_map()
test_ml_oh = multi_label_one_hot (test_tags, test_label_map)

print test_ml_oh

In [ ]:
"""
This function will read and store training images in a pickle file
The input is a dataframe that contains the image filename and corresponding labels
The pickle file will contain both the image data and the label in two distinct arrays
"""

def pickle_train_images(train_image_root_dir, df, pickle_filename, image_resize = (64, 64)):
   
    features = []
    labels = []
   
    for index, row in df.iterrows():
        
        image_name = row['image_name']
        tags = row['tags']
        
        print image_name, tags

        image_file_name = os.path.join(train_image_root_dir, image_name + '.jpg')

        image = read_image(image_file_name, image_resize)
        features.append(image)
        labels.append(multi_label_one_hot(tags, label_map))
            
    print 'pickle_filename = ', pickle_filename
    pickle.dump((features, labels), open(pickle_filename, 'wb'))

In [ ]:
"""
This function will read and store test images  in a pickle file
This code will go by the list of input filenames and store the image in a pickle file
The pickle file will contain both the image data and the image file name in two distinct arrays
"""



def pickle_test_images(filenames, pickle_filename, image_resize = (64, 64)):
    
    test_features = []
    for image_filename in filenames:
        
        print 'image_filename = ', image_filename

        image = read_image(image_filename, image_resize)
        
        test_features.append(image)


    print 'pickle_filename = ', pickle_filename
    
    pickle.dump((filenames, test_features), open(pickle_filename, 'wb'))
          

In [ ]:
"""
This function will read and store training images  in batches in a pickle file
The input is a pickle batch size - which is different from training batch size
This will create a batch of filenames and repeatedly call pickle_train_images for each batch
"""


#### Pickle batch size is different from training batch size

def make_train_batches(label_map, pickle_batch_size, image_resize = (64, 64), testing_limit=None):
    
    #print 'image_resize=', image_resize, 'train_valid_split=', train_valid_split
    
    df = pd.read_csv(train_csv_file)
    
    print 'total training images = ', len(df )
     
    df = df[:testing_limit]  # This is a quick-hack to unit test this function
      
    train = df
    
    print 'total training images = ', len(train )
         
    # first pickle pre-processed training images - in batches determined by pickle_batch_size
    batch_id = 0
    for start in range(0, len(train), pickle_batch_size):
        end = min(start + pickle_batch_size, len(train))
                        
        pickle_train_images(train_root_dir, train[start:end], 'train_batch.{}.p'.format(batch_id))
        batch_id += 1
        
#unit-test          
make_train_batches(label_map, 3, testing_limit = 10)

In [ ]:
"""
generators for batches of image data and corresponding labels
"""


def batch_features_labels(features, labels, batch_size):
    """
    Split features and labels into batches
    """
    for start in range(0, len(features), batch_size):
        end = min(start + batch_size, len(features))
        yield features[start:end], labels[start:end]


"""
This routine will load a training set from pickle file
This will then generate the training  batches out of the data
"""

def load_preprocess_training_batch(batch_id, batch_size):
    """
    Load the Preprocessed Training data and return them in batches of <batch_size> or less
    """
    filename = 'train_batch.{}.p'.format(batch_id)
    print filename
    features, labels = pickle.load(open(filename, mode='rb'))
    #print 'feature=', features, "labels=", labels

    # Return the training data in batches of size <batch_size> or less
    return batch_features_labels(features, labels, batch_size)


In [ ]:
"""
This function will read and store test images  in batches in a pickle file
The input is a pickle batch size - which is different from training batch size
This will read images from test and additional test directories
This will create pickle file for each batch determined by  pickle_batch_size
"""
#### Pickle batch size is different from training batch size

def make_test_batches(pickle_batch_size, image_resize = (64, 64), testing_limit = None):
   
    data_root_folder = os.path.abspath("../data/")
    dir_path = os.path.join(data_root_folder, 'test-jpg')

    file_names = os.listdir(dir_path)
    
    print 'number of files = ', len(file_names)
    
    test_filepaths = []
    for filename in file_names:
        test_filepaths.append(os.path.join(dir_path, filename))
    
    # now for jpg_additional
    dir_path = os.path.join(data_root_folder, 'test-jpg-additional')
    
    file_names = os.listdir(dir_path)
    
    print 'number of files = ', len(file_names)
    
    for filename in file_names:
        test_filepaths.append(os.path.join(dir_path, filename))
        
    print 'total testing images = ', len(test_filepaths )
    
    # some hack for testing
    test_filepaths = test_filepaths[:testing_limit]
    
    print 'total testing images = ', len(test_filepaths )
         
    #  pickle pre-processed test images - in batches determined by pickle_batch_size
    batch_id = 0
    for start in range(0, len(test_filepaths), pickle_batch_size):
        end = min(start + pickle_batch_size, len(test_filepaths))
                        
        pickle_test_images(test_filepaths[start:end], 'test_batch.{}.p'.format(batch_id))
        batch_id += 1



# unit-test          
make_test_batches(3, testing_limit = 10)


In [31]:
"""
Run this cell to pre-process all training data
"""

# pickle batch size = 1000
label_map = make_label_map()

make_train_batches(label_map, pickle_batch_size=1000, image_resize = (64, 64))

total training images =  40479
total training images =  40479
train_0 haze primary
/Users/amitabhac/Kaggle/data/train-jpg/train_0.jpg
train_1 agriculture clear primary water
/Users/amitabhac/Kaggle/data/train-jpg/train_1.jpg
train_2 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_2.jpg
train_3 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_3.jpg
train_4 agriculture clear habitation primary road
/Users/amitabhac/Kaggle/data/train-jpg/train_4.jpg
train_5 haze primary water
/Users/amitabhac/Kaggle/data/train-jpg/train_5.jpg
train_6 agriculture clear cultivation primary water
/Users/amitabhac/Kaggle/data/train-jpg/train_6.jpg
train_7 haze primary
/Users/amitabhac/Kaggle/data/train-jpg/train_7.jpg
train_8 agriculture clear cultivation primary
/Users/amitabhac/Kaggle/data/train-jpg/train_8.jpg
train_9 agriculture clear cultivation primary road
/Users/amitabhac/Kaggle/data/train-jpg/train_9.jpg
train_10 agriculture clear primary slash_burn water
/Users/amitabhac/Kaggle/

train_124 bare_ground clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_124.jpg
train_125 agriculture clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_125.jpg
train_126 agriculture clear primary road
/Users/amitabhac/Kaggle/data/train-jpg/train_126.jpg
train_127 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_127.jpg
train_128 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_128.jpg
train_129 agriculture clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_129.jpg
train_130 partly_cloudy primary
/Users/amitabhac/Kaggle/data/train-jpg/train_130.jpg
train_131 agriculture clear primary road water
/Users/amitabhac/Kaggle/data/train-jpg/train_131.jpg
train_132 haze primary
/Users/amitabhac/Kaggle/data/train-jpg/train_132.jpg
train_133 haze primary
/Users/amitabhac/Kaggle/data/train-jpg/train_133.jpg
train_134 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_134.jpg
train_135 clear primary water
/Users/amitabhac/Kaggle/data/train-jpg/tr

train_243 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_243.jpg
train_244 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_244.jpg
train_245 clear primary blooming
/Users/amitabhac/Kaggle/data/train-jpg/train_245.jpg
train_246 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_246.jpg
train_247 agriculture clear cultivation habitation primary road selective_logging
/Users/amitabhac/Kaggle/data/train-jpg/train_247.jpg
train_248 haze primary
/Users/amitabhac/Kaggle/data/train-jpg/train_248.jpg
train_249 partly_cloudy primary
/Users/amitabhac/Kaggle/data/train-jpg/train_249.jpg
train_250 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_250.jpg
train_251 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_251.jpg
train_252 clear habitation primary
/Users/amitabhac/Kaggle/data/train-jpg/train_252.jpg
train_253 agriculture clear habitation primary road
/Users/amitabhac/Kaggle/data/train-jpg/train_253.jpg
train_254 clear primary
/Users/amitabhac

train_363 haze primary water
/Users/amitabhac/Kaggle/data/train-jpg/train_363.jpg
train_364 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_364.jpg
train_365 partly_cloudy primary
/Users/amitabhac/Kaggle/data/train-jpg/train_365.jpg
train_366 clear primary water
/Users/amitabhac/Kaggle/data/train-jpg/train_366.jpg
train_367 haze primary
/Users/amitabhac/Kaggle/data/train-jpg/train_367.jpg
train_368 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_368.jpg
train_369 agriculture clear cultivation habitation primary
/Users/amitabhac/Kaggle/data/train-jpg/train_369.jpg
train_370 agriculture clear habitation primary road
/Users/amitabhac/Kaggle/data/train-jpg/train_370.jpg
train_371 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_371.jpg
train_372 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_372.jpg
train_373 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_373.jpg
train_374 agriculture clear primary
/Users/amitabhac/Kaggle/data/train-j

/Users/amitabhac/Kaggle/data/train-jpg/train_482.jpg
train_483 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_483.jpg
train_484 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_484.jpg
train_485 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_485.jpg
train_486 bare_ground clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_486.jpg
train_487 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_487.jpg
train_488 clear primary water
/Users/amitabhac/Kaggle/data/train-jpg/train_488.jpg
train_489 clear primary water
/Users/amitabhac/Kaggle/data/train-jpg/train_489.jpg
train_490 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_490.jpg
train_491 clear habitation primary road water
/Users/amitabhac/Kaggle/data/train-jpg/train_491.jpg
train_492 partly_cloudy primary
/Users/amitabhac/Kaggle/data/train-jpg/train_492.jpg
train_493 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_493.jpg
train_494 clear primary
/Users/amitabhac/Kaggl

...

train_2955 agriculture clear primary water
/Users/amitabhac/Kaggle/data/train-jpg/train_2955.jpg
train_2956 agriculture clear primary road
/Users/amitabhac/Kaggle/data/train-jpg/train_2956.jpg
train_2957 clear primary road water
/Users/amitabhac/Kaggle/data/train-jpg/train_2957.jpg
train_2958 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_2958.jpg
train_2959 partly_cloudy primary road
/Users/amitabhac/Kaggle/data/train-jpg/train_2959.jpg
train_2960 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_2960.jpg
train_2961 cloudy
/Users/amitabhac/Kaggle/data/train-jpg/train_2961.jpg
train_2962 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_2962.jpg
train_2963 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_2963.jpg
train_2964 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_2964.jpg
train_2965 cloudy
/Users/amitabhac/Kaggle/data/train-jpg/train_2965.jpg
train_2966 agriculture partly_cloudy primary road
/Users/amitabhac/Kaggle/data/train-j

train_3121 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_3121.jpg
train_3122 agriculture clear cultivation primary
/Users/amitabhac/Kaggle/data/train-jpg/train_3122.jpg
train_3123 agriculture clear cultivation primary
/Users/amitabhac/Kaggle/data/train-jpg/train_3123.jpg
train_3124 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_3124.jpg
train_3125 cloudy
/Users/amitabhac/Kaggle/data/train-jpg/train_3125.jpg
train_3126 cloudy
/Users/amitabhac/Kaggle/data/train-jpg/train_3126.jpg
train_3127 agriculture partly_cloudy primary water
/Users/amitabhac/Kaggle/data/train-jpg/train_3127.jpg
train_3128 cloudy
/Users/amitabhac/Kaggle/data/train-jpg/train_3128.jpg
train_3129 haze primary
/Users/amitabhac/Kaggle/data/train-jpg/train_3129.jpg
train_3130 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_3130.jpg
train_3131 partly_cloudy primary
/Users/amitabhac/Kaggle/data/train-jpg/train_3131.jpg
train_3132 partly_cloudy primary
/Users/amitabhac/Kaggle/data/train-jpg

train_3239 clear primary road
/Users/amitabhac/Kaggle/data/train-jpg/train_3239.jpg
train_3240 haze primary
/Users/amitabhac/Kaggle/data/train-jpg/train_3240.jpg
train_3241 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_3241.jpg
train_3242 partly_cloudy primary
/Users/amitabhac/Kaggle/data/train-jpg/train_3242.jpg
train_3243 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_3243.jpg
train_3244 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_3244.jpg
train_3245 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_3245.jpg
train_3246 agriculture partly_cloudy primary
/Users/amitabhac/Kaggle/data/train-jpg/train_3246.jpg
train_3247 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_3247.jpg
train_3248 agriculture clear primary road
/Users/amitabhac/Kaggle/data/train-jpg/train_3248.jpg
train_3249 clear habitation primary road water
/Users/amitabhac/Kaggle/data/train-jpg/train_3249.jpg
train_3250 agriculture haze primary water
/Users/amitabhac/K

train_3361 agriculture clear primary road
/Users/amitabhac/Kaggle/data/train-jpg/train_3361.jpg
train_3362 agriculture clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_3362.jpg
train_3363 agriculture partly_cloudy primary
/Users/amitabhac/Kaggle/data/train-jpg/train_3363.jpg
train_3364 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_3364.jpg
train_3365 haze primary
/Users/amitabhac/Kaggle/data/train-jpg/train_3365.jpg
train_3366 cloudy
/Users/amitabhac/Kaggle/data/train-jpg/train_3366.jpg
train_3367 clear cultivation habitation primary
/Users/amitabhac/Kaggle/data/train-jpg/train_3367.jpg
train_3368 cloudy
/Users/amitabhac/Kaggle/data/train-jpg/train_3368.jpg
train_3369 partly_cloudy primary
/Users/amitabhac/Kaggle/data/train-jpg/train_3369.jpg
train_3370 clear primary
/Users/amitabhac/Kaggle/data/train-jpg/train_3370.jpg
train_3371 partly_cloudy primary
/Users/amitabhac/Kaggle/data/train-jpg/train_3371.jpg
train_3372 clear primary road water
/Users/amitabhac/Kaggl

KeyboardInterrupt: 

In [ ]:
"""
Run this cell to pre-process all test data
"""

# pickle batch size = 1000

make_test_batches(pickle_batch_size=1000, image_resize = (64, 64))